### Link: https://www.mysqltutorial.org/mysql-join/

In [1]:
import pandas as pd

import mysql.connector
 
# Connecting from the server
conn = mysql.connector.connect(user = 'root',
                               host = 'localhost',
                              database = 'mysql_db',
                              password = 'pass@learning')

cursor = conn.cursor()

#### Creating dummy tables

In [6]:
cursor.execute("""DROP TABLE if exists members""")

cursor.execute("""CREATE TABLE members (
    member_id INT AUTO_INCREMENT,
    name VARCHAR(100),
    PRIMARY KEY (member_id)
)""")

cursor.execute("""INSERT INTO members(name)
VALUES('John'),('Jane'),('Mary'),('David'),('Amelia')""")

conn.commit()

In [7]:
cursor.execute("""DROP TABLE if exists committees""")

cursor.execute("""CREATE TABLE committees (
    committee_id INT AUTO_INCREMENT,
    name VARCHAR(100),
    PRIMARY KEY (committee_id)
)""")

cursor.execute("""INSERT INTO committees(name)
VALUES('John'),('Mary'),('Amelia'),('Joe')""")

conn.commit()

In [9]:
pd.read_sql_query("""select * from members
                  """ \
                  , conn)

,member_id,name
0,1,John
1,2,Jane
2,3,Mary
3,4,David
4,5,Amelia


In [10]:
pd.read_sql_query("""select * from committees
                  """ \
                  , conn)

,committee_id,name
0,1,John
1,2,Mary
2,3,Amelia
3,4,Joe


#### Inner Join

In [11]:
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
INNER JOIN committees c ON c.name = m.name
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1,John,1,John
1,3,Mary,2,Mary
2,5,Amelia,3,Amelia


In [12]:
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
INNER JOIN committees c USING(name)
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1,John,1,John
1,3,Mary,2,Mary
2,5,Amelia,3,Amelia


#### Left Join

In [13]:
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
LEFT JOIN committees c USING(name)
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1,John,1.0,John
1,2,Jane,NaN,None
2,3,Mary,2.0,Mary
3,4,David,NaN,None
4,5,Amelia,3.0,Amelia


#### Application of filters in different ways to get different result

In [23]:
# If filter on left/right table is added in "on condition" then left table will be returned completely but 
# data from right table won't come for those records where above mentioned filter condition is false 
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
LEFT JOIN committees c
on m.name = c.name and m.member_id < 5
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1,John,1.0,John
1,2,Jane,NaN,None
2,3,Mary,2.0,Mary
3,4,David,NaN,None
4,5,Amelia,NaN,None


In [26]:
# If filter on left/right table is added in "on condition" then left table will be returned completely but 
# data from right table won't come for those records where above mentioned filter condition is false 
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
LEFT JOIN committees c
on m.name = c.name and c.committee_id > 1
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1,John,NaN,None
1,2,Jane,NaN,None
2,3,Mary,2.0,Mary
3,4,David,NaN,None
4,5,Amelia,3.0,Amelia


In [27]:
# If filter on left/right table is added in "on condition" then left table will be returned completely but 
# data from right table won't come for those records where above mentioned filter condition is false 
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
LEFT JOIN committees c
on m.name = c.name and m.member_id <> 3 and c.committee_id > 1
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1,John,NaN,None
1,2,Jane,NaN,None
2,3,Mary,NaN,None
3,4,David,NaN,None
4,5,Amelia,3.0,Amelia


In [29]:
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    (select * from members where member_id < 5) m
LEFT JOIN 
(select * from committees where committee_id < 4) c
on m.name = c.name
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1,John,1.0,John
1,2,Jane,NaN,None
2,3,Mary,2.0,Mary
3,4,David,NaN,None


In [14]:
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
LEFT JOIN committees c USING(name)
WHERE c.committee_id IS NULL
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,2,Jane,None,None
1,4,David,None,None


In [38]:
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
LEFT JOIN committees c USING(name)
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1,John,1.0,John
1,2,Jane,NaN,None
2,3,Mary,2.0,Mary
3,4,David,NaN,None
4,5,Amelia,3.0,Amelia


In [42]:
# 
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
LEFT JOIN committees c USING(name)
WHERE m.member_id < 5 and ifnull(c.committee_id, 2) > 1
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,2,Jane,NaN,None
1,3,Mary,2.0,Mary
2,4,David,NaN,None


#### Right Join

In [15]:
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
RIGHT JOIN committees c on c.name = m.name
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1.0,John,1,John
1,3.0,Mary,2,Mary
2,5.0,Amelia,3,Amelia
3,NaN,None,4,Joe


In [16]:
# Right join and left join swapping the tables giving same result
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    committees c
LEFT JOIN members m USING(name)
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1.0,John,1,John
1,3.0,Mary,2,Mary
2,5.0,Amelia,3,Amelia
3,NaN,None,4,Joe


#### Cross Join

In [17]:
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
CROSS JOIN committees c
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1,John,4,Joe
1,1,John,3,Amelia
2,1,John,2,Mary
3,1,John,1,John
4,2,Jane,4,Joe
5,2,Jane,3,Amelia
6,2,Jane,2,Mary
7,2,Jane,1,John
8,3,Mary,4,Joe
9,3,Mary,3,Amelia


In [20]:
# Cross join and inner join without on clause returns same result
pd.read_sql_query("""SELECT 
    m.member_id, 
    m.name AS member, 
    c.committee_id, 
    c.name AS committee
FROM
    members m
inner JOIN committees c
                  """ \
                  , conn)

,member_id,member,committee_id,committee
0,1,John,4,Joe
1,1,John,3,Amelia
2,1,John,2,Mary
3,1,John,1,John
4,2,Jane,4,Joe
5,2,Jane,3,Amelia
6,2,Jane,2,Mary
7,2,Jane,1,John
8,3,Mary,4,Joe
9,3,Mary,3,Amelia
